# Celeribity Recognition using Amazon Rekognition


This notebook provides a walkthrough of [celebrity recognition API](https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html) in Amazon Rekognition. You can quickly identify well known people in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

bucketName = bucket

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and draw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir -p m1tmp
tempFolder = 'm1tmp/'

# Recognize celebrities in image
***

In [ ]:
imageName = "content-moderation/media/GrandTourjc.png"

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Call Rekognition to recognize celebrities in the image

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html

recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': imageName,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Name, Id, Urls and additional information about 
# their facial attributes.

display(recognizeCelebritiesResponse)

#### Show image with bounding boxes around recognized celebrities

In [ ]:
# Define a function that will display image with bounded boxes around recognized celebrites
# We will call this function in next step
  
def drawBoundingBoxes (sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    # Download image locally
    imageLocation = tempFolder+os.path.basename(sourceImage)
    s3.download_file(bucketName, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
# Extract bounding box information from JSON response above and display image with bounding boxes around celebrites.

boxes = []
celebrities = recognizeCelebritiesResponse['CelebrityFaces']
for celebrity in celebrities:
    boxes.append ((celebrity['Name'], celebrity['Face']['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes)

# Recognize celebrities in video
 Celebrity recognition in video is an async operation. 
 https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
 - We first start celebrity recognition job which returns a Job Id.
 - We can then call `get_celebrity_recognition` to get the job status and after job is complete, we can get celebrity metadata.
 - In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
***

In [ ]:
videoName = "content-moderation/media/GrandTour720.mp4"

#### Call Rekognition to start a job for celebrity rekognition

In [ ]:
# Start celebrity recognition job
startCelebrityRekognition = rekognition.start_celebrity_recognition(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

celebrityJobId = startCelebrityRekognition['JobId']
display("Job Id: {0}".format(celebrityJobId))

#### Wait for celebrity rekognition job to complete

In [ ]:
%%time

# Wait for celebrity recognition job to complete
# In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP'
)

while(getCelebrityRecognition['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP')
    
display(getCelebrityRecognition['JobStatus'])

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see list Celebrities which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Timestamp, Name, Id, Urls
# and additional information about their facial attributes.

display(getCelebrityRecognition)

#### Dislpay names of recognized celebrities in the video

In [ ]:
theCelebs = {}

# Display timestamps and celebrites detected at that time
strDetail = "Celebrites detected in video<br>=======================================<br>"
strOverall = "Celebrities in the overall video:<br>=======================================<br>"

# Celebrities detected in each frame
for celebrity in getCelebrityRecognition['Celebrities']:
    if 'Celebrity' in celebrity :
        cconfidence = celebrity["Celebrity"]["Confidence"]
        if(cconfidence > 95):
            ts = celebrity ["Timestamp"]
            cname = celebrity["Celebrity"]["Name"]
            strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, cname, round(cconfidence,2))
            if not cname in theCelebs:
                theCelebs[cname] = cname
            

# Unique faces detected in video
for theCeleb in theCelebs:
    strOverall = strOverall + "Name: {}<br>".format(theCeleb)

# Display results
display(HTML(strOverall))
#display(HTML(strDetail))

#### Show video in the player

In [ ]:
# Show video in a player

s3FilePrefix = "https://s3.amazonaws.com"
if(not awsRegion == 'us-east-1'):
    s3FilePrefix = "https://s3-{}.amazonaws.com".format(awsRegion)

s3VideoUrl = "{0}/{1}/{2}".format(s3FilePrefix, bucketName, videoName)

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td><td>{}</td></tr></table>".format(videoTag, strDetail)

display(HTML(videoui))

## Try recognizing custom celebrities 
***

In [ ]:
# Now let us try an image with non-celebrities in the image.

customCelebrityImageName = "content-moderation/media/serverless-bytes.png"

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': customCelebrityImageName})))

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image

customCelebrityResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': customCelebrityImageName,
        }
    }
)

In [ ]:
# Display Rekognition response
# You will see Rekognition return an empty list for CelebrityFaces and 
# UnrecognizedFaces list with unrecognized faces that were detected in the image.
# In the next module you will learn how to get custom-celebrity faces recognized.

display(customCelebrityResponse)

In [ ]:
#Show image and bounded boxes around detected faces

# Extract BB info from response
cboxes = []
faces = customCelebrityResponse['UnrecognizedFaces']
for face in faces:
    cboxes.append (('Unrecognized Face', face['BoundingBox']))
    
drawBoundingBoxes(customCelebrityImageName, cboxes)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetCelebrityRecognition.html

***

You have successfully used Amazon Rekognition to identify celebrities in images an videos. In the next module, Recognize Custom Celebrities, you will learn how to recognize your custom celebrities in the images and videos.

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}